## In this Notebook I will start with the very Basics of RNN's and Build all the way to latest deep learning architectures to solve NLP problems. It will cover the Following:

### Simple RNN's
### Word Embeddings : Definition and How to get them
### LSTM's
### GRU's
### BI-Directional RNN's
### Encoder-Decoder Models (Seq2Seq Models)
### Attention Models
### Transformers - Attention is all you need
### BERT

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
# Import our dataset 
train = pd.read_csv('E:/Ai Notebooks/Datasets/Jigshaw dataset/train.csv')
test = pd.read_csv('E:/Ai Notebooks/Datasets/Jigshaw dataset/test.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
# We will do binary classification and we will take sample dataset:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'], axis=1, inplace=True)

In [4]:
train.shape

(159571, 3)

In [6]:
train = train.loc[:12000,:]
train.shape

(12001, 3)

In [8]:
# Getting max number of words coz this will help us in padding later 
train['comment_text'].apply(lambda x: len(str(x).split())).max()

1403

In [26]:
# Writing a function to getting AUC score for validation: 
# Understand ROC vs AUC score 
def auc(pred, target):
    fpr, tpr, threshold = metrics.roc_curve(target, pred)
    return (metrics.auc(fpr, tpr))

In [11]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values, test_size = 0.2, random_state = 42, stratify= train.toxic.values, shuffle = True)

## Simple RNN:
Recurrent Neural Network(RNN) are a type of Neural Network where the output from previous step are fed as input to the current step. In traditional neural networks, all the inputs and outputs are independent of each other, but in cases like when it is required to predict the next word of a sentence, the previous words are required and hence there is a need to remember the previous words. Thus RNN came into existence, which solved this issue with the help of a Hidden Layer.

Why RNN's?

https://www.quora.com/Why-do-we-use-an-RNN-instead-of-a-simple-neural-network

In [18]:
# Using Keras Tokenizer:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
token = Tokenizer()
max_len = 1403
token.fit_on_texts(list(xtrain) + list(xvalid))

In [19]:
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

In [20]:
word_index = token.word_index

In [22]:
# RNN model 
model = Sequential()
model.add(Embedding(len(word_index)+1, 300, input_length=1403))
model.add(SimpleRNN(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1403, 300)         13049100  
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 100)               40100     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 13,089,301
Trainable params: 13,089,301
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(xtrain_pad, ytrain, epochs = 5)

C:\Users\Dream Walker\anaconda3\envs\Ai\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
9600/9600 [==============================] - 311s 32ms/step - loss: 0.2830 - accuracy: 0.9119
Epoch 2/5
9600/9600 [==============================] - 311s 32ms/step - loss: 0.1595 - accuracy: 0.9464
Epoch 3/5
9600/9600 [==============================] - 321s 33ms/step - loss: 0.0986 - accuracy: 0.9673
Epoch 4/5
9600/9600 [==============================] - 323s 34ms/step - loss: 0.0484 - accuracy: 0.9847
Epoch 5/5
9600/9600 [==============================] - 321s 33ms/step - loss: 0.0239 - accuracy: 0.9931


In [27]:
scores = model.predict(xvalid_pad)
auc(scores, yvalid)

0.8816874637789819

### Code Explanation:
Tokenization

So if you have watched the videos and referred to the links, you would know that in an RNN we input a sentence word by word. We represent every word as one hot vectors of dimensions : Numbers of words in Vocab +1.
What keras Tokenizer does is , it takes all the unique words in the corpus,forms a dictionary with words as keys and their number of occurences as values,it then sorts the dictionary in descending order of counts. It then assigns the first value 1 , second value 2 and so on. So let's suppose word 'the' occured the most in the corpus then it will assigned index 1 and vector representing 'the' would be a one-hot vector with value 1 at position 1 and rest zereos.
Try printing first 2 elements of xtrain_seq you will see every word is represented as a digit now.

In [ ]:
xtrain_seq[:1]

We have done PADDING to make sequance equal inputs:
#### Building the Neural Network
To understand the Dimensions of input and output given to RNN in keras her is a beautiful article : https://medium.com/@shivajbd/understanding-input-and-output-shape-in-lstm-keras-c501ee95c65e

The first line model.Sequential() tells keras that we will be building our network sequentially . Then we first add the Embedding layer. Embedding layer is also a layer of neurons which takes in as input the nth dimensional one hot vector of every word and converts it into 300 dimensional vector , it gives us word embeddings similar to word2vec. We could have used word2vec but the embeddings layer learns during training to enhance the embeddings. Next we add an 100 RNN units without any dropout or regularization At last we add a single neuron with sigmoid function which takes output from 100 RNN cells (Please note we have 100 LSTM cells not layers) to predict the results and then we compile the model using adam optimizer..

### Comments on the model

We can see our model achieves an accuracy of 1 which is just insane , we are clearly overfitting I know , but this was the simplest model of all ,we can tune a lot of hyperparameters like RNN units, we can do batch normalization , dropouts etc to get better result. The point is we got an AUC score of 0.82 without much efforts and we know have learnt about RNN's .Deep learning is really revolutionary

## Word Embeddings: 

While building our simple RNN models we talked about using word-embeddings , So what is word-embeddings and how do we get word-embeddings? Here is the answer :

https://www.coursera.org/learn/nlp-sequence-models/lecture/6Oq70/word-representation
https://machinelearningmastery.com/what-are-word-embeddings/

The latest approach to getting word Embeddings is using pretained GLoVe or using Fasttext. Without going into too much details, I would explain how to create sentence vectors and how can we use them to create a machine learning model on top of it and since I am a fan of GloVe vectors, word2vec and fasttext. In this Notebook, I'll be using the GloVe vectors. You can download the GloVe vectors from here http://www-nlp.stanford.edu/data/glove.840B.300d.zip or you can search for GloVe in datasets on Kaggle and add the file

In [30]:
# Load Glove model in Dictionary: 
glove = {}
f = open('E:/Ai Notebooks/Datasets/glove/glove.6B.300d.txt', 'r', encoding='utf-8')
for line in f:
    line = line.split(' ')
    word = line[0]
    coeff = np.asarray([float(val) for val in line[1:]])
    glove[word] = coeff
f.close()

In [40]:
# Create Embedding metrics for words we have in dataset: 
embedding_metrics = np.zeros((len(word_index)+1, 300))
for word, i in word_index.items():
    embedding = glove.get(word)
    if embedding is not None:
        embedding_metrics[i] = embedding

## LSTM's:

Simple RNN's were certainly better than classical ML algorithms and gave state of the art results, but it failed to capture long term dependencies that is present in sentences . So in 1998-99 LSTM's were introduced to counter to these drawbacks.

#### Code Implementation
We have already tokenized and paded our text for input to LSTM's



In [44]:
# LSTM model 
model = Sequential()
model.add(Embedding(len(word_index)+1, 300, weights=[embedding_metrics], input_length=1403, trainable=False))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1403, 300)         13049100  
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 13,209,601
Trainable params: 160,501
Non-trainable params: 13,049,100
_________________________________________________________________


In [46]:
model.fit(xtrain_pad, ytrain, epochs = 5)

Epoch 1/5
9600/9600 [==============================] - 885s 92ms/step - loss: 0.2156 - accuracy: 0.9257
Epoch 2/5
9600/9600 [==============================] - 856s 89ms/step - loss: 0.1681 - accuracy: 0.9420
Epoch 3/5
9600/9600 [==============================] - 879s 92ms/step - loss: 0.1463 - accuracy: 0.9479
Epoch 4/5
9600/9600 [==============================] - 822s 86ms/step - loss: 0.1327 - accuracy: 0.9521
Epoch 5/5
9600/9600 [==============================] - 871s 91ms/step - loss: 0.1291 - accuracy: 0.9535


In [47]:
scores = model.predict(xvalid_pad)
auc(scores, yvalid)

0.964285569546381

### Code Explanation
As a first step we calculate embedding matrix for our vocabulary from the pretrained GLoVe vectors . Then while building the embedding layer we pass Embedding Matrix as weights to the layer instead of training it over Vocabulary and thus we pass trainable = False. Rest of the model is same as before except we have replaced the SimpleRNN By LSTM Units

Comments on the Model
We now see that the model is not overfitting and achieves an auc score of 0.96 which is quite commendable , also we close in on the gap between accuracy and auc . We see that in this case we used dropout and prevented overfitting the data

## GRU's
### Basic Overview
Introduced by Cho, et al. in 2014, GRU (Gated Recurrent Unit) aims to solve the vanishing gradient problem which comes with a standard recurrent neural network. GRU's are a variation on the LSTM because both are designed similarly and, in some cases, produce equally excellent results . GRU's were designed to be simpler and faster than LSTM's and in most cases produce equally good results and thus there is no clear winner.

In [49]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_metrics],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])   
    
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1403, 300)         13049100  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1403, 300)         0         
_________________________________________________________________
gru_1 (GRU)                  (None, 300)               540900    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 301       
Total params: 13,590,301
Trainable params: 541,201
Non-trainable params: 13,049,100
_________________________________________________________________


In [50]:
model.fit(xtrain_pad, ytrain, epochs = 5)

Epoch 1/5
9600/9600 [==============================] - 55631s 6s/step - loss: 0.2100 - accuracy: 0.9310ETA: 1:23:06 - loss: 0.2149 - accuracy: 0.928 - ETA: 1:19:36 - loss
Epoch 2/5
9600/9600 [==============================] - 8990s 936ms/step - loss: 0.1288 - accuracy: 0.9545
Epoch 3/5
9600/9600 [==============================] - 11117s 1s/step - loss: 0.1157 - accuracy: 0.9574
Epoch 4/5
9600/9600 [==============================] - 9352s 974ms/step - loss: 0.0955 - accuracy: 0.9645
Epoch 5/5
9600/9600 [==============================] - 9948s 1s/step - loss: 0.0796 - accuracy: 0.9717: 2:12 - loss: 0.0792 - accuracy:


In [51]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (auc(scores,yvalid)))

Auc: 0.96%


## Bi-Directional RNN's:
Code is same as before,only we have added bidirectional nature to the LSTM cells we used before and is self explanatory. We have achieve similar accuracy and auc score as before and now we have learned all the types of typical RNN architectures

In [53]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_metrics],
                     input_length=max_len,
                     trainable=False))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1403, 300)         13049100  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 600)               1442400   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 601       
Total params: 14,492,101
Trainable params: 1,443,001
Non-trainable params: 13,049,100
_________________________________________________________________


In [56]:
# IT takes more time so i did not implement this